In [1]:
import os, os.path,sys
import requests
import csv
from tqdm import tqdm
import time

import collections
csv.field_size_limit(sys.maxsize)
import pandas as pd
import numpy as np



In [2]:
tracks_file = './data/tracks.csv'
complete_tracks_file = './data/tracks_complete.csv'
failed_tracks_file = './data/tracks_failed.csv'

In [3]:
def get_rows(file_name):
    with open(file_name) as infile:
        file_size = sum(1 for row in infile) 
    return file_size    

def get_track_info(artist,track):
    base_url = 'http://ws.audioscrobbler.com/2.0/'
    context = '?method=track.getInfo&api_key=db3a905ecd9f0b7d0e9641c39e7527dd&format=json'
    params = '&artist={0}&track={1}'.format(artist,track)
    resp = requests.get(base_url+context+params)
    if resp.status_code != 200:
        print('EXCEPTION: ',resp.status_code)
#        print(resp.json())
        raise Exception('Failed')
    response = resp.json()
    duration = response['track']['duration']
    tags = response['track']['toptags']['tag']
    genre = [x['name'] for x in tags]
    genre = ','.join(genre)
    return duration,genre    

In [4]:
# artist = 'iron & wine'
# track = 'such great heights'
# get_track_info(artist,track)

In [5]:
file_size = get_rows(tracks_file)
file_size

1550240

In [6]:
columns = ['artist_id','artist_name','track_name','count']

In [7]:
tracks = pd.read_csv(tracks_file,names=columns)
tracks.head(5)

,artist_id,artist_name,track_name,count
0,299278d3-25dd-4f30-bae4-5b571c28034d,the postal service,such great heights,3991
1,8a26ca9b-d542-449b-a5e7-224da9eb8a77,boy division,love will tear us apart,3651
2,a74b1b7f-71a5-4011-9441-d0b5e4122711,radiohead,karma police,3533
3,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,death cab for cutie,soul meets body,3479
4,9c9f1380-2516-4fc9-a3e6-f9f61941d090,muse,supermassive black hole,3463


In [8]:
def process_tracks(tracks,file_size,complete_tracks_file,failed_tracks_file):
        
    try:
        os.remove(complete_tracks_file) 
        os.remove(failed_tracks_file) 
    except OSError:
        pass

    i = 0
    with open(complete_tracks_file, 'w') as writer:
        writer = csv.writer(writer,quoting=csv.QUOTE_NONNUMERIC)
        
        for row in tqdm(tracks.itertuples(),total=file_size):
            data = []
            try:
                artist_id = row.artist_id
                artist = row.artist_name
                track = row.track_name
                count = row.count

                data.append(artist_id)
                data.append(artist)
                data.append(track)
                data.append(count)
                
                duration,genre = get_track_info(artist,track)
                data.append(duration)
                data.append(genre)
                writer.writerow(data)
            except Exception:
                if os.path.exists(failed_tracks_file):
                    append_write = 'a' 
                else:
                    append_write = 'w+' 
                
                with open(failed_tracks_file, append_write) as failed:
                    failed = csv.writer(failed,quoting=csv.QUOTE_NONNUMERIC)
                    failed.writerow(data)
                pass
            
#             if(i==1000):
#                 break
            i = i+1 
#            time.sleep(0.1)
    print('processing complete!')            

In [9]:
process_tracks(tracks,file_size,complete_tracks_file,failed_tracks_file)

  3%|▎         | 44510/1550240 [1:51:23<236:58:54,  1.76it/s] 

EXCEPTION:  500


  9%|▊         | 132710/1550240 [5:40:29<234:04:06,  1.68it/s] 

EXCEPTION:  500


 11%|█▏        | 176728/1550240 [7:34:02<275:12:24,  1.39it/s] 

...

EXCEPTION:  500


 43%|████▎     | 672425/1550240 [28:39:50<134:41:25,  1.81it/s]

EXCEPTION:  500


 45%|████▌     | 697885/1550240 [29:47:17<29:36:33,  8.00it/s] 

KeyboardInterrupt: 